In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `make importdb`, load it here
# (2) save links to ~/Downloads/links.txt in onetab format and run this script (or otherwise modify the CSV as desired)
# (3) Upload back up with `make csv_to_s3`
# (4) Run `heroku run make importdb` to get it to production
links = pd.read_csv('data/export.csv')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""""""""""""""&nbsp;&ldquo;People disagree about whet...",moralitylab.bc.edu,2016-12-05 23:00:00-06,2016-12-05 23:00:00-06,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""""""""""""""&nbsp;&ldquo;Why give globally? Why giv...",www.jefftk.com,2016-12-06 23:00:00-06,2016-12-06 23:00:00-06,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""""""""""""""&nbsp;&ldquo;I&rsquo;m fascinated by pe...",calnewport.com,2016-12-07 23:00:00-06,2016-12-07 23:00:00-06,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""""""""""""""&nbsp;&ldquo;In Glen&rsquo;s voting mec...",www.freakonomics.com,2016-12-08 23:00:00-06,2016-12-08 23:00:00-06,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here&rsqu...,"""""""""""""""&nbsp;&ldquo;August 13\, 6:40 PM: I&rsq...",www.wired.com,2016-12-09 23:00:00-06,2016-12-09 23:00:00-06,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
74702,130389,https://twitter.com/ProfPaulPoast/status/14062...,"Paul Poast on Twitter: ""The Emancipation Procl...",\N,\N,2021-08-03 11:50:57.387258,2021-08-03 11:50:57.387269,\N,\N,Custom,67,0
74703,130390,https://twitter.com/michael_nielsen/status/140...,"Michael Nielsen on Twitter: ""There is nothing ...",\N,\N,2021-08-03 11:50:57.398099,2021-08-03 11:50:57.398147,\N,\N,Custom,3,0
74704,130391,https://twitter.com/Brian_Riedl/status/1406665...,"Brian Riedl 🧀 on Twitter: ""Some ""Twitter is no...",\N,\N,2021-08-03 11:50:57.408605,2021-08-03 11:50:57.408619,\N,\N,Custom,71,0
74705,130392,https://twitter.com/deepfates/status/140664079...,"invisible college dropout on Twitter: ""if you ...",\N,\N,2021-08-03 11:50:57.420035,2021-08-03 11:50:57.420047,\N,\N,Custom,11,0


In [2]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

73076    128763
73077    128764
73078    128765
73079    128766
73080    128767
          ...  
74702    130389
74703    130390
74704    130391
74705    130392
74706    130393
Name: id, Length: 1631, dtype: int64

In [3]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [4]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 100 == 0:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/945
...101/945
...201/945
...301/945
...401/945
...501/945
...601/945
...701/945
...801/945
...901/945


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""""""""""""""&nbsp;&ldquo;People disagree about whet...",moralitylab.bc.edu,2016-12-05 23:00:00-06,2016-12-05 23:00:00-06,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""""""""""""""&nbsp;&ldquo;Why give globally? Why giv...",www.jefftk.com,2016-12-06 23:00:00-06,2016-12-06 23:00:00-06,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""""""""""""""&nbsp;&ldquo;I&rsquo;m fascinated by pe...",calnewport.com,2016-12-07 23:00:00-06,2016-12-07 23:00:00-06,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""""""""""""""&nbsp;&ldquo;In Glen&rsquo;s voting mec...",www.freakonomics.com,2016-12-08 23:00:00-06,2016-12-08 23:00:00-06,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here&rsqu...,"""""""""""""""&nbsp;&ldquo;August 13\, 6:40 PM: I&rsq...",www.wired.com,2016-12-09 23:00:00-06,2016-12-09 23:00:00-06,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
75643,131330,https://eeassoc.s3.amazonaws.com/s3fs-public/2...,How Much Can We Generalize From Impact Evaluat...,\N,\N,2021-08-27 22:04:33.955344,2021-08-27 22:04:33.955358,\N,\N,Custom,93,0
75644,131331,https://en.wikipedia.org/wiki/Timeline_of_huma...,Timeline of human evolution - Wikipedia,\N,\N,2021-08-27 22:04:33.966262,2021-08-27 22:04:33.966275,\N,\N,Custom,97,0
75645,131332,https://www.vox.com/22589042/covid-pandemic-pr...,Does Congress know what it would take to stop ...,\N,\N,2021-08-27 22:04:33.977441,2021-08-27 22:04:33.977454,\N,\N,Custom,37,0
75646,131333,https://guzey.com/personal/funding/,People and Things I Would Fund If I Were a Bil...,\N,\N,2021-08-27 22:04:33.987719,2021-08-27 22:04:33.987732,\N,\N,Custom,39,0


In [5]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
73076,128763,https://www.cs.utexas.edu/users/dahlin/bookshe...,Dr. R. W. Hamming's Advice on Research,\N,\N,2021-08-03 10:19:48.010140,2021-08-03 10:19:48.010157,\N,\N,Custom,86,0
73077,128764,https://autotranslucence.wordpress.com/2018/03...,Becoming a magician – Autotranslucence,\N,\N,2021-08-03 10:19:48.022198,2021-08-03 10:19:48.022222,\N,\N,Custom,93,0
73078,128765,https://www.sup.org/books/title/?id=26137,The Case for U.S. Nuclear Weapons in the 21st ...,\N,\N,2021-08-03 10:19:48.034344,2021-08-03 10:19:48.034396,\N,\N,Custom,17,0
73079,128766,https://www.belfercenter.org/sites/default/fil...,untitled,\N,\N,2021-08-03 10:19:48.049316,2021-08-03 10:19:48.049333,\N,\N,Custom,45,0
73080,128767,https://forecastapp.substack.com/p/whos-best-a...,Who's Best at Predicting the Future? - by Jare...,\N,\N,2021-08-03 10:19:48.062958,2021-08-03 10:19:48.062972,\N,\N,Custom,84,0
...,...,...,...,...,...,...,...,...,...,...,...,...
75643,131330,https://eeassoc.s3.amazonaws.com/s3fs-public/2...,How Much Can We Generalize From Impact Evaluat...,\N,\N,2021-08-27 22:04:33.955344,2021-08-27 22:04:33.955358,\N,\N,Custom,93,0
75644,131331,https://en.wikipedia.org/wiki/Timeline_of_huma...,Timeline of human evolution - Wikipedia,\N,\N,2021-08-27 22:04:33.966262,2021-08-27 22:04:33.966275,\N,\N,Custom,97,0
75645,131332,https://www.vox.com/22589042/covid-pandemic-pr...,Does Congress know what it would take to stop ...,\N,\N,2021-08-27 22:04:33.977441,2021-08-27 22:04:33.977454,\N,\N,Custom,37,0
75646,131333,https://guzey.com/personal/funding/,People and Things I Would Fund If I Were a Bil...,\N,\N,2021-08-27 22:04:33.987719,2021-08-27 22:04:33.987732,\N,\N,Custom,39,0


In [6]:
links['id'] = links['id'].astype(int)
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)

In [7]:
links.to_csv('data/export.csv', index=False)